# Create predcitors with Prophet and DeepAR+
* **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Hyperparameters

In [5]:
forecastHorizon = 13 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 13 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "W"

A detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)


#### Predictor algorithms

In [6]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [7]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + target_suffix + suffix

In [8]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )




### DeepAR+

In [9]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix

In [10]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )





## Check if the predictors are active


In [11]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [12]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
target_prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]

In [13]:
target_deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [ ]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= target_prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_predictorArn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_I

## Predictor evaluation

#### Prophet

In [22]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=target_prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 93215.56052267776,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.03502681337900551,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06350647181067595,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.028907054457241888,
                                                                                          'Quantile': 0.1}]}},
                                           

In [23]:
# deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
# forecast.describe_predictor(PredictorArn= deepar_predictorArn)

#### DeepAR+


In [24]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 141566.86632568153,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.04185169785133818,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.1028758994052715,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.059267123872013106,
                                                                                          'Quantile': 0.1}]}},
                                      

In [25]:
%store target_prophet_predictorArn
%store target_deepar_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
